# Jonathan Halverson
# Wednesday, October 25, 2017
# Geron Chapter 12: Intro to Tensorflow

Let's import the module and then create a simple computation graph:

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(9, name="y")
f = x * x + x * y + 4

The graph has been created. Note that no computation has taken place as this point. Next we create a TF session and execute the graph:

In [2]:
with tf.Session() as sess:
     x.initializer.run()
     y.initializer.run()
     result = f.eval()

In [3]:
print(result)

40


Note that x.initializer.run() is equivalent to tf.get_default_session().run(x.initializer) and f.eval() is equivalent to calling tf.get_default_session.run(f).

In [4]:
type(x)

tensorflow.python.ops.variables.Variable

In [5]:
type(f)

tensorflow.python.framework.ops.Tensor

In [6]:
type(result)

numpy.int32

### Graph 2

In [7]:
tf.reset_default_graph()

Let's try another graph:

In [8]:
a = tf.Variable(-10.0, name='a')
b = tf.Variable(1e3, name='b')
g = a / b - 1.0
init = tf.global_variables_initializer()

In [9]:
with tf.Session() as sess:
     init.run()
     result = g.eval()

In [10]:
print(result)

-1.01


### Interactive sessions

In [11]:
tf.reset_default_graph()

In [12]:
u = tf.Variable(4, name='u')
v = tf.Variable(7, name='v')
h = v * u

In [13]:
sess = tf.InteractiveSession()
u.initializer.run()
v.initializer.run()
result = sess.run(h)
print(result)
sess.close()

28


One needs to explicitly close the session as is done above.

### Managing graphs

In [14]:
tf.reset_default_graph()

In [15]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [16]:
graph = tf.Graph()
with graph.as_default():
     x2 = tf.Variable(2)

In [17]:
x2.graph is graph

True

In [18]:
x2.graph is tf.get_default_graph()

False

Any node that you create is automatically added to the default graph. You have explicitly create another graph and then make this the default to assign nodes to that graph.

In [19]:
with tf.get_default_graph().as_default():
     x3 = tf.Variable(8, name='x3')

In [20]:
type(tf.get_default_graph())

tensorflow.python.framework.ops.Graph

### Lifecycle of a node

In [21]:
tf.reset_default_graph()

In [22]:
w = tf.constant(5)
x = w + 2
y = x + 5
z = x + 3

In [23]:
with tf.Session() as sess:
     print(y.eval())
     print(z.eval())

12
10


When z is evaluated the DAG is re-traversed. This is inefficient so we can use another way:

In [24]:
with tf.Session() as sess:
     y_val, z_val = sess.run([y, z])
     print(y_val)
     print(z_val)

12
10


### Working with tensors

In [25]:
tf.reset_default_graph()

In [26]:
import numpy as np

x = tf.Variable(np.arange(10.0, 20.0, 1.0))
y = tf.Variable(np.random.rand(10), dtype=tf.float64)
z = x + y
init = tf.global_variables_initializer()

In [27]:
sess = tf.InteractiveSession()
init.run()
print(z.eval())
sess.close()

[ 10.12265896  11.42715627  12.32208546  13.63749184  14.00903344
  15.21084087  16.77584747  17.78543502  18.75289412  19.57654744]


### Linear regression with tensorflow (the normal equation)

In [28]:
f = '../machine_learning/geron_housing/housing.csv'

import pandas as pd
data = pd.read_csv(f, header=0)
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
longitude             20640 non-null float64
latitude              20640 non-null float64
housing_median_age    20640 non-null float64
total_rooms           20640 non-null float64
total_bedrooms        20433 non-null float64
population            20640 non-null float64
households            20640 non-null float64
median_income         20640 non-null float64
median_house_value    20640 non-null float64
ocean_proximity       20640 non-null object
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [30]:
housing_labels = data.median_house_value.copy()
data.drop(['ocean_proximity', 'median_house_value'], axis=1, inplace=True)

In [31]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler

# impute missing data
imp = Imputer(strategy='median')
housing = imp.fit_transform(data)

# add bias term
housing = np.c_[np.ones((housing.shape[0], 1)), housing]

# standardize the features
#std_sc = StandardScaler()
#housing = std_sc.fit_transform(housing)

In [32]:
m, n = housing.shape
m, n

(20640, 9)

In [33]:
housing_labels.values.reshape(-1, 1)

array([[ 452600.],
       [ 358500.],
       [ 352100.],
       ..., 
       [  92300.],
       [  84700.],
       [  89400.]])

In [34]:
housing

array([[  1.00000000e+00,  -1.22230000e+02,   3.78800000e+01, ...,
          3.22000000e+02,   1.26000000e+02,   8.32520000e+00],
       [  1.00000000e+00,  -1.22220000e+02,   3.78600000e+01, ...,
          2.40100000e+03,   1.13800000e+03,   8.30140000e+00],
       [  1.00000000e+00,  -1.22240000e+02,   3.78500000e+01, ...,
          4.96000000e+02,   1.77000000e+02,   7.25740000e+00],
       ..., 
       [  1.00000000e+00,  -1.21220000e+02,   3.94300000e+01, ...,
          1.00700000e+03,   4.33000000e+02,   1.70000000e+00],
       [  1.00000000e+00,  -1.21320000e+02,   3.94300000e+01, ...,
          7.41000000e+02,   3.49000000e+02,   1.86720000e+00],
       [  1.00000000e+00,  -1.21240000e+02,   3.93700000e+01, ...,
          1.38700000e+03,   5.30000000e+02,   2.38860000e+00]])

Begin to create the graph:

In [35]:
X = tf.constant(housing, name="X", dtype=tf.float32)
y = tf.constant(housing_labels.values.reshape(-1, 1), name="y", dtype=tf.float32)
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [36]:
with tf.Session() as sess:
     theta_value = theta.eval()

In [37]:
print(theta_value)

[[ -3.62829275e+06]
 [ -4.32717031e+04]
 [ -4.30518516e+04]
 [  1.13395923e+03]
 [ -6.47900295e+00]
 [  8.14451599e+01]
 [ -3.98782883e+01]
 [  7.81404572e+01]
 [  3.96517695e+04]]


In [38]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression().fit(housing[:, 1:], housing_labels)
print lin_reg.intercept_, lin_reg.coef_

-3570118.06149 [ -4.26104026e+04  -4.24754782e+04   1.14445085e+03  -6.62091740e+00
   8.11609666e+01  -3.98732002e+01   7.93047225e+01   3.97522237e+04]


### Linear regression with tensorflow (the gradient descent)

In [39]:
tf.reset_default_graph()

In [40]:
epochs = 1000
learning_rate = 0.01

In [42]:
X = tf.constant(housing, name="X", dtype=tf.float32)
y = tf.constant(housing_labels.values.reshape(-1, 1), name="y", dtype=tf.float32)
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name='theta')
y_pred = tf.matmul(X, theta, name='predictions')
error = y - y_pred
mse = tf.reduce_mean(tf.square(error), name='mse')
gradients = (2.0 / m) * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

ValueError: Dimensions must be equal, but are 9 and 10 for 'predictions' (op: 'MatMul') with input shapes: [20640,9], [10,1].

In [ ]:
init = tf.global_variables_initializer()

The graph is now complete. Now we start a session to execute the computation graph:

In [ ]:
with tf.Session() as sess:
     sess.run(init)
     for epoch in range(epochs):
          if (epoch % 100 == 0):
               print('Epoch', epoch, "MSE=", mse.eval())
          sess.run(training_op)
     best_theta = theta.eval()